![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

# import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.



In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [ ]:
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
df_test.info()

In [ ]:
df_test.head(5)

In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.info()

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [ ]:
data.sample(5)

In [ ]:
data.Reviews[1]

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

# Cleaning and Prepping Data
Обычно данные содержат в себе кучу мусора, который необходимо почистить, для того чтобы привести их в приемлемый формат. Чистка данных — это необходимый этап решения почти любой реальной задачи.   
![](https://analyticsindiamag.com/wp-content/uploads/2018/01/data-cleaning.png)

In [ ]:
# Определим количество пропусков в каждом признаке.

data.isna().sum()

Пропуски содержат признаки Cuisine Style (23%), Price Range (35%), Number of Reviews (6%). Целевая переменная Rating пропусков не содержит.

In [ ]:
# Оформим названия признаков в едином стиле для удобства дальнейшей работы.

data.columns = ['_'.join(col.split()).lower() for col in data.columns]
data.columns

In [ ]:
# Выберем случайный образец данных из таблицы и посмотрим на него.

data.sample(10)

Признак **cuisine_style** в действительности описывает не только тип кухни по страновому признаку (например, Japanese или Italian), но и типы блюд (Suschi, Sopups) или формат ресторана (Fast Food, Cafe) Возможно, это удастся использовать в дальнейшем.

In [ ]:
#Определим уникальные значения целевой переменной и ее распределение.

data.rating.value_counts()

In [ ]:
#Определим вид и тип содержимого признака reviews.

print(data.reviews[2])
print(type(data.reviews[2]))

In [ ]:
#Определим вид и тип содержимого признака url_ta.

print(data.url_ta[2])
print(type(data.url_ta[2]))
print(data.url_ta[5])
print(type(data.url_ta[5]))

В этом признаке содержится указание на район расположения ресторана, возможно, это будет влиять на его рейтинг.

### Работа с признаками

#### restaurant_id

In [ ]:
data.restaurant_id.value_counts()

Видим, что есть значительное количество повторов, то есть ID ресторанов не уникальны. Это может быть связано как с ошибками при заполнении таблицы, так и с тем, что ресторан - сетевой. Тем более, что раньше при просмотра признака Cuisine Style мы видели указания на то, что ресторан может быть сетевым (формат заведения Fast Food, Cafe).

In [ ]:
# Посмотрим примеры ресторанов с одинаковым ID.

data[(data.restaurant_id == 'id_633')].sample(10)

Видим много совпадений в признаках ranking и price_range, однако описания кухни очень разнятся. Нельзя сказать однозначно что одинаковый ID имеют именно сетевые рестораны, но это утверждение можно оставить в качестве предположения.

In [ ]:
# Создадим дополнительный признак Сетевой ресторан (chain_rest).
# Значения: 1 - сетевой ресторан, 0 - несетевой ресторан.

chain_list = list(data.restaurant_id.value_counts()[data.restaurant_id.value_counts() > 1].index)
data['chain_rest'] = data[data.restaurant_id.isin(chain_list)].restaurant_id.apply(lambda x: 1)
data['chain_rest'].fillna(0, inplace=True)

data.head()

In [ ]:
#Создан новый признак chain_rest. Проверим его уникальные значения.

data.chain_rest.value_counts()

Пропусков и случайных неуникальных значений в признаке нет. Большая часть ресторанов по нашим предположениям - сетевые.

#### city

In [ ]:
#Посмотрим на список городов, представленный в признаке и заодно на его распределение.

data.city.value_counts()


Видим, что большинство ресторанов расположены в наиболее посещаемых туристических столицах мира (что соответсвует тематике сайта, который дает советы путешественникам). При этом самые популярные города не всегда совпадают со столицами стран (Милан опережает Рим). Кроме того, есть странное название города Oportо. Википедия говорит нам, что в португальском языке есть два варианта написания названия города Порту - Oporto и Porto, повторов нет, поэтому исправлять мы его не будем.

In [ ]:
#Создадим признак, указывающий на то, является ли город столицей (capital).

cap_list = ['London', 'Paris', 'Madrid', 'Berlin', 'Rome', 'Prague', 'Lisbon', 'Vienna', 'Amsterdam', 'Brussels', 
           'Stockholm', 'Budapest', 'Budapest', 'Dublin', 'Copenhagen', 'Edinburgh', 'Oslo', 'Helsinki', 'Bratislava',
           'Luxembourg', 'Ljubljana']
data['capital'] = data[data.city.isin(cap_list)].city.apply(lambda x: 1)
data['capital'].fillna(0, inplace=True)
data.head()

In [ ]:
#Создан новый признак capital. Проверим его уникальные значения.

data.capital.value_counts()

На рейтинг ресторана могут влиять экономические показатели - доходы, получаемые от туризма и внутренний валовый продукт. Первый показатель говорит нам о том, какое количество денег могут потратить на посещение ресторана туристы, второй - о том, насколько потенциально высок уровень цен в стране. Данные получены из открытых источников и приводятся по состоянию на 2019 год.

In [ ]:
#Добавим признак country.
#Для этого создадим сначала справочник городов и стран.

country_dict = {
    'Paris': 'France',
    'Stockholm': 'Sweden',
    'London': 'UK',
    'Berlin': 'Germany', 
    'Munich': 'Germany',
    'Oporto': 'Portugal', 
    'Milan': 'Italy',
    'Bratislava': 'Slovakia',
    'Vienna': 'Austria', 
    'Rome': 'Italy',
    'Barcelona': 'Spain',
    'Madrid': 'Spain',
    'Dublin': 'Ireland',
    'Brussels': 'Belgium',
    'Zurich': 'Switzerland',
    'Warsaw': 'Poland',
    'Budapest': 'Hungary', 
    'Copenhagen': 'Denmark',
    'Amsterdam': 'Netherlands',
    'Lyon': 'France',
    'Hamburg': 'Germany', 
    'Lisbon': 'Portugal',
    'Prague': 'Czech',
    'Oslo': 'Norway', 
    'Helsinki': 'Finland',
    'Edinburgh': 'UK',
    'Geneva': 'Switzerland',
    'Ljubljana': 'Slovenia',
    'Athens': 'Greece',
    'Luxembourg': 'Luxembourg',
    'Krakow': 'Poland'       
}

data['country'] = data.city.map(country_dict)

data.country.value_counts()

Видим, что подавляющее число ресторанов расположено в Великобритании, Испании, Франции.

In [ ]:
#Проверим новый признак на наличие пропусков.

data.country.isna().sum()

Пропусков нет, значит, мы определили страну для каждого города. Страна будет маркером для распределения экономических данных.

In [ ]:
#Добавим признак tourism_income.
#Для этого создадим сначала справочник стран и доходов от туризма (в миллионах долларов).

tourism_dict = {
    'Austria': 22979,
    'Belgium': 13474,
    'Czech': 7451,
    'Denmark': 8420, 
    'Finland': 3607,
    'France': 67370, 
    'Germany': 42977,
    'Greece': 19029,
    'Hungary': 6930, 
    'Ireland': 6185,
    'Italy': 49262,
    'Luxembourg': 4990,
    'Netherlands': 18641,
    'Norway': 5672,
    'Poland': 14042,
    'Portugal': 19621,
    'Slovakia': 3200, 
    'Slovenia': 3194,
    'Spain': 73765,
    'Sweden': 14977,
    'Switzerland': 17042, 
    'UK': 51882     
}

data['tourism_income'] = data.country.map(tourism_dict)

#Проверим новый признак на наличие пропусков.

data.tourism_income.isna().sum()


Пропусков нет, значит, каждому городу присвоена страна. Проверим на примере правильность распределения городов по странам.

In [ ]:
data.sample(10)

Все верно. Продолжаем создание новых признаков на основании экономических данных - ВВП.

In [ ]:
#Добавим признак gdp.
#Для этого создадим сначала справочник стран и ВВП на душу населения (в долларах).

gdp_dict = {
    'Austria': 52813,
    'Belgium': 49912,
    'Czech': 39511,
    'Denmark': 53449, 
    'Finland': 48098,
    'France': 47322, 
    'Germany': 54874,
    'Greece': 30501,
    'Hungary': 33033, 
    'Ireland': 83001,
    'Italy': 40923,
    'Luxembourg': 113550,
    'Netherlands': 58095,
    'Norway': 76243,
    'Poland': 33072,
    'Portugal': 33211,
    'Slovakia': 36878, 
    'Slovenia': 38343,
    'Spain': 41998,
    'Sweden': 54666,
    'Switzerland': 65077, 
    'UK': 46870     
}

data['gdp'] = data.country.map(gdp_dict)

#Проверим новый признак на наличие пропусков.

data.gdp.isna().sum()

Можно предположить, что какую-то роль в ценообразовании и рейтинге ресторана играет соотношение туристических доходов и уровня жизни в стране. Создадим еще один новый признак - отношение доходов от туризма к ВВП на душу населения. Назовем его tour_inc_gdp.

In [ ]:
data['tour_inc_gdp'] = (data.tourism_income / data.gdp).round(2)

In [ ]:
#Посмотрим на распределение нового признака в датасете.

plt.figure(figsize=(10,4))
data.tour_inc_gdp.value_counts(ascending=False).plot(kind='bar')


Мы видим, что большинство ресторанов находятся в странах, где доходы от туризма первышают ВВП на душу населения, то есть в туристических центрах.

Визуализируем отношение признака tour_inc_gdp и рейтинга ресторанов.

In [ ]:
data['tour_inc_gdp'][data['rating'] > 3].hist(bins=100)

Мы видим, что высокий рейтинг (более 3 пунктов) имеют рестораны с высоким доходом от туризма.

#### cuisine_style

Ранее мы видели, что в признаке cuisine_style имеется 23% пропусков. К сожалению, прямой корреляции между городом, страной и типом кухни нет (Будапешт - венгерская), поэтому имеющиеся пробелы мы заполним строковым выражением. Перед этим создадим колонку, в которой отразим наличие пропусков в признаке.

In [ ]:
#Отражаем наличие пропусков в исходных данных и заполняем их в текущем признаке.

data['cuisine_style_NAN'] = data['cuisine_style'].isna().astype('uint8')
data['cuisine_style'].fillna("['Other']", inplace=True)

Количество кухонь может влиять на рейтинг, поэтому добавляем этот признак. Там, где кухня не определена (замена пропусков), количество будем считать равным 1 (в любом ресторане представлена по крайней мере 1 кухня, никакие другие признаки не дают нам возможность сделать заполнение более точным).

In [ ]:
type(data['cuisine_style'][2])

In [ ]:
#Преобразуем строковые данные с названиями кухонь в список
data['cuisine_style_list'] = data['cuisine_style'].str.findall(r"'(\b.*?\b)'")

#Заполним пропуски 0
data['cuisine_style_list'] = data['cuisine_style_list'].fillna(0)

In [ ]:
type(data['cuisine_style_list'][2])


In [ ]:
data['cuisine_style_list'][2]

In [ ]:
#Создадим новый признак - количество кухонь в ресторане (cuisine_count).
data['cuisine_count'] = data['cuisine_style_list'].apply(lambda x: 1 if x == 0 else len(x))

In [ ]:
#Просматриваем уникальные значения и распределения нового признака.

data.cuisine_count.value_counts()

Учитывая то, что в исходном датасете в признаке cuisine_style пропуски заменяли более 23%, которые мы затем заменили на 1, понятно, почему в большинстве ресторанов представлен один вид кухни.
Посмотрим, какие виды кухонь встречаются реже всего, чтобы определить редкие и создать признак для количества редких кухонь в ресторане.

In [ ]:
plt.figure(figsize=(18,6))
data.explode('cuisine_style_list')['cuisine_style_list'].value_counts(ascending=False).plot(kind='bar');

In [ ]:
#Чтобы правильно определить, какие кухни считать редкими, выясним медиану частотности разных видов.
print((data.explode('cuisine_style_list')['cuisine_style_list'].value_counts(ascending=False)).median())

In [ ]:
rare_list = data.explode('cuisine_style_list')['cuisine_style_list'].value_counts()[
    data.explode('cuisine_style_list')['cuisine_style_list'].value_counts() < 200].index.tolist()

In [ ]:
len(rare_list)

In [ ]:
data.explode('cuisine_style_list')['cuisine_style_list'].value_counts(ascending=False)

Также составим список самых популярных кухонь, которые есть почти в каждом ресторане.

In [ ]:
popular_list = data.explode('cuisine_style_list')['cuisine_style_list'].value_counts()[
    data.explode('cuisine_style_list')['cuisine_style_list'].value_counts() > 5900].index.tolist()

In [ ]:
popular_list

#### price_range

In [ ]:
# уникальные значения признака price_range
data['price_range'].unique()

In [ ]:
data['price_range'].value_counts(ascending=False)

In [ ]:
#Замена строковых значений признака price_range

data['price_range'] = data['price_range'].fillna(0)

price_dict = {'$':1, '$$ - $$$':2, '$$$$':3}
data['price_range'] = data['price_range'].replace(to_replace=price_dict)
data['price_range'].unique()

Первая приблизительная замена выполнена. Теперь заполним пропуски в price_range более точно, для этого выясним среднее значение этого признака по городам.

In [ ]:
# 1 Получаем датафрейм, в котором сгруппированы только города и рейтинг цен
price_city =  data[['city','price_range']]

# 2 Удаляем пропущенные значения из датафрейма
price_city = price_city.loc[price_city.ne(0).all(axis=1)]

# 3 Получаем среднее значение по городам, учитывающее ненулевой рейтинг

price_city = price_city.groupby(['city'])[['price_range']].mean().round(2)

# 4 Получаем словарь со средними значениями

price_range_dict = price_city['price_range'].to_dict()
price_range_dict

In [ ]:
#Заменяем пропуски средним значением по городам
x = data['city'].where(data['price_range']==0).replace(to_replace=price_range_dict)
data['price_range'].where(data['price_range']!=0, other=x, inplace=True)
data['price_range'].unique()

#### number_of_revievs

In [ ]:
#Смотрим, есть ли варианты, где в числе ревю ничего не стоит, а на самом деле ревю и даты есть

data['number_of_reviews'] = data['number_of_reviews'].fillna(0)
data[data['number_of_reviews'] == 0]

Мы видим, что есть ситуации, когда ревю якобы нет, но на самом деле они есть, об этом нам говорит вывод по группировке. Создаем переменную, в которую загружаем строку, обозначающую, что ревю нет.

In [ ]:
#Словарь со значением среднего количества ревю по городам

# 1 Получаем датафрейм, в котором сгруппированы только города и рейтинг цен
number_reviews = data[['city','number_of_reviews']]

# 2 Удаляем пропущенные значения из датафрейма
number_reviews = number_reviews.dropna(axis = 0)

# 3 Получаем среднее значение по городам, учитывающее ненулевое количество ревю

number_reviews = number_reviews.groupby(['city'])[['number_of_reviews']].mean().round(0)

# 4 Получаем словарь со средними значениями

number_reviews_dict = number_reviews['number_of_reviews'].to_dict()
number_reviews_dict

In [ ]:
#Переменная со строковым выражением, обозначающим фактическое отсутствие ревю
no_rew = data.iloc[7][6]
no_rew

In [ ]:
#Добавляем признак is_review
data['is_review']=0
data['is_review'].where(data['reviews']==no_rew,other=1, inplace=True)

In [ ]:
data.head(10)

In [ ]:
#Заполняем пропуски в количестве ревю средним количеством ревю по городам

y = data[data['number_of_reviews'] == 0]['city'].where(data['is_review']==1).replace(to_replace=number_reviews_dict)
data['number_of_reviews']=data['number_of_reviews'].where(data['number_of_reviews']!=0, other=y)
data['number_of_reviews']=data['number_of_reviews'].fillna(0)
data.head()

#### reviews

In [ ]:
#Посмотрим на содержание ячейки с этим признаком.

print(data.iloc[1][6])

In [ ]:
type(data.iloc[1][6])

Мы видим, что в ячейке есть дата первого и последнего отзыва. Разницу в днях можно выделить как отдельный признак.

In [ ]:
#Создадим сначала колонки с первой и последней датой отзывов.

import numpy as np
import re
import datetime as dt
def reviews_date(rew, count):
    date = re.findall(r'\d\d?/\d\d?/\d+', str(rew))
    if len(date) == 0:
        return np.nan
    if count == 1:
        return pd.to_datetime(date[0])
    elif len(date) == 2:
        return pd.to_datetime(date[1])
    else:
        return np.nan

data['reviews_first'] = data['reviews'].apply(lambda x: reviews_date(x, 1))
data['reviews_second'] = data['reviews'].apply(lambda x: reviews_date(x, 2))

In [ ]:
data['reviews_first'] = data['reviews_first'].fillna(data['reviews_first'].min())
data['reviews_second'] = data['reviews_second'].fillna(data['reviews_second'].max())
data.head()

In [ ]:
# Найдем разницу между датами

data['time_delta'] = (data['reviews_second'] - data['reviews_first']).dt.days
data['time_delta'] = data['time_delta'].apply(lambda x: x*-1 if x < 0 else x)

data.head()

Посмотрим, есть ли взаимосвязь между количеством ревю (отзывов) и частотой, с которой их оставляют.

In [ ]:
data[['number_of_reviews','time_delta']].sample(15)

Видим, что довольно часто видна обратная взаимосвязь - чем больше отзывов, тем чаще их оставляют. Отразим это в новом признаке - коэффициент частотности (frequency).

In [ ]:
data['frequency'] = (data['number_of_reviews']/data['time_delta']).round(2)


#### url_ta

Посмотрим на содержимое ячеек.

In [ ]:
print(data.iloc[3][7])
print(data.iloc[16][7])
print(data.iloc[2][7])
print(data.iloc[22][7])

Как мы можем видеть, иногда в описании ресторана есть указание на район, в котором располагается ресторан. Расположение также может влиять на рейтинг, однако указание района есть не во всех ячейках и нет регулярного паттерна, который помог бы его обнаружить (всегда на одном и том же месте, отделяется определенным знаком и т.д.) Поэтому от этого признака я предлагаю избавиться.

In [ ]:
data = data.drop(['url_ta'], axis = 1)

#### id_ta

In [ ]:
data['id_ta'] = data['id_ta'].apply(lambda x: int(x[1:]))

In [ ]:
data.head(10)

### Dummies

In [ ]:
data = pd.get_dummies(data, columns=['city'])

In [ ]:
data = data.join(pd.get_dummies(data['cuisine_style_list'].apply(pd.Series).stack()).sum(level=0))

In [ ]:
data.columns

Мы видим, что появилось множество признаков с названиями редких кухонь и самых популярных кухонь, которые не могут иметь большого внимания на целевую переменную (популярные кухни есть везде, редкие кухни относятся к небольшому количеству ресторанов). Поэтому возьмем названия редких и популярных кухонь из словарей, которые мы создали при работе cuisine_style и удалим эти признаки по списку.

In [ ]:
data = data.drop(rare_list, axis = 1)

In [ ]:
data = data.drop(popular_list, axis = 1)

In [ ]:
data.columns

### Удаление нечисловых признаков

In [ ]:
#Смотрим, какие признаки включают в себя объектные данные

data.select_dtypes(include = ['object']).columns

In [ ]:
#Создадим список признаков с объектными данными для удаления

drop_list = ['restaurant_id', 'cuisine_style', 'reviews', 'country',
       'cuisine_style_list']


In [ ]:
#Удаляем объектные признаки

data = data.drop(drop_list, axis = 1)

In [ ]:
# Удаляем даты, поскольку из этих данных мы уже получили частоту отзывов.

data = data.drop(['reviews_first'], axis = 1)
data = data.drop(['reviews_second'], axis = 1)

In [ ]:
data.info(verbose=True)

### Распределение целевой переменной

In [ ]:
plt.figure(figsize=(10, 5))
data['rating'].value_counts(ascending=True).plot(kind='barh')

In [ ]:
plt.rcParams['figure.figsize'] = (15,11)
sns.heatmap(data.corr(), cmap='vlag')

### Корреляция с целевой переменной

In [ ]:
data.corr().rating.sort_values(ascending=False).to_frame()

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [ ]:
# на всякий случай, заново подгружаем данные
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

In [ ]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['Restaurant_id','ID_TA',], axis = 1, inplace=True)
    
    
    # ################### 2. NAN ############################################################## 
    # Далее заполняем пропуски, вы можете попробовать заполнением средним или средним по городу и тд...
    df_output['Number of Reviews'].fillna(0, inplace=True)
    # тут ваш код по обработке NAN
    # ....
    
    
    # ################### 3. Encoding ############################################################## 
    # для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
    df_output = pd.get_dummies(df_output, columns=[ 'City',], dummy_na=True)
    # тут ваш код не Encoding фитчей
    # ....
    
    
    # ################### 4. Feature Engineering ####################################################
    # тут ваш код не генерацию новых фитчей
    # ....
    
    
    # ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    
    return df_output

>По хорошему, можно было бы перевести эту большую функцию в класс и разбить на подфункции (согласно ООП). 

#### Запускаем и проверяем что получилось

In [ ]:
df_preproc = preproc_data(data)
df_preproc.sample(10)

In [ ]:
df_preproc.info()

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
